**Note:** This notebook gives further clarification on the functions and parameters of `qrml`, however for full details, please refer to the source code or the paper [[1]](#1). 

In [1]:
import qrml
import numpy                as np
import matplotlib.pyplot    as plt

%matplotlib qt

### Pointcloud

We take a closer look at `qrml` applied to the real projective plane $\mathbb{RP}^2$. We take the standard embedding into $\mathbb{R}^4$ given by:
$$ S^2\to \mathbb{R}^4 : (x, y, z) \mapsto (xy, xz, y^2 − z^2, 2yz)$$
We recall that the real projective plane can be contructed from the 2-D closed disk $D^2$ by identifying antipodal points on its boundary under the standard quotient topology. We see whether these identifications can recovered by `qrml`.

We generate our pointcloud by uniformly sampling 1000 points from $S^2$ (found in `../datasets/sphere.txt`) and applying the above embedding .

In [2]:
sphere = np.loadtxt('../datasets/sphere.txt')
x0 = sphere[:, 0] * sphere[:, 1]
x1 = sphere[:, 0] * sphere[:, 2]
x2 = sphere[:, 1]**2 - sphere[:, 2]**2
x3 = 2 * sphere[:, 1] * sphere[:, 2]
rp2 = np.stack([x0, x1, x2, x3], axis=-1)

### Projection

We follow the same steps as in the README.md file to compute a 1-skeleton on our data and its projection.

In [3]:
params = {'S1':0.2, 'k':10, 'threshold_var':0.08, 'edge_sen':1, 'k0':100}

S_rp2 = qrml.Simplex()
S_rp2.build_simplex(rp2, **params)
S_rp2.normal_coords(two_d=False, **params)

Restricted license - for non-production use only - expires 2023-10-25


Full explanations of the parameters `k`, `threshold_var` and `edge_sen` can be found in the paper [[2]](#2) under the "naive" algorithm; these parameters control the construction of the 1-skeleton on our data under `build_simplex`. From this 1-skeleton, we compute the estimated dimension of our pointcloud.

Roughly, `k` is the parameter for the KNN 1-skeleton that we start with and that we refine through "visible" then "safe" edges. The parameters `threshold_var` and `edge_sen` control the quality of the edges that we take. Higher values of `threshold_var` and `edge_sen` result in more edges being kept. The standard values the paper [[2]](#2) recommends are `k=10, threshold_var=0.08, edge_sen=1`.

To estimate the intrinsic dimension of our pointcloud, for each point, we take the set of its neighbouring points (in the computed 1-skeleton) and apply PCA. We take the local dimension of this neighbourhood to be the "knee" point of the curve given by `PCA.explained_variance_ratio_` from `scikit-learn`. We use the function `KneeLocator` from the python package `kneed`, which takes the parameter `S1`, to locate the "knee" point; we take `S1=0.2` as default. The overall estimated dimension is then the mode of all the local estimated dimensions.

Our implementation introduces the extra parameter `k0` for the computation of the projection under `normal_coords`. We found this necessary in order to make the algorithm to work. Originally, in order to compute the projection of a generic point, we find the shortest path from the base point (of our projection) to our generic point by Dijkstra's algorithm applied to the 1-skeleton. We take the neighbouring points of the predecessor, on this path, to the generic point, whose projections are already computed, and use this local information to compute the projection of the generic point. We need at least the estimated dimension number of neighbouring points to compute the projection.

We found in implementing this algorithm that we would regularly run into points, throughout the pointcloud, which do not have enough neighbouring points to correctly compute the projection. To correct this, we instead take the set of neighbouring points (which already have their projection computed) of the neighbouring points (which already have their projection computed) of the predecessor point (we exclude the predecessor point) and take the estimated dimension plus `k0` points which are closest to our generic point, to compute each projection. Hence, `k0` controls how local we require our local data to be, in computing projections.

Note that when `two_d=True`, we force the estimated dimension of our pointcloud to be two - otherwise, we use the method given above. We provide this option as our quotient functions require that the projection we use is two-dimensional.

### Visualization

The method `show_boundary` allows us to visualize our projection, the boundary of our projection, how much projected points are displaced and how boundary points are connected to other boundary points. We require that the dimension of our projection is two. We note that `c` provides the colour map for the projected points.

In [4]:
S_rp2.show_boundary(alpha=1, tol=2, c=rp2[:, 0], show_tear_points=True, a=2.5, show_connections=True, show_pointcloud=False, connection_tol=5, **params)

In order to compute the boundary of our projection, we compute the boundary of the alpha shape [[3]](#3) of our projected points with the parameter `alpha`. The implementation of alpha shape, that we use, is taken from [[4]](#4). We require that the boundary of the alpha shape is a single 1-cycle. We place an orientation on this boundary.

With this boundary, we "clean" it by removing non-significant loops. If a point is connected, in the 1-skeleton to another boundary point which is between two and `tol` points away inclusive, we then find the largest loop in this range and remove the intermediate points from the boundary. We repeat this process looping over the remaining boundary points until no more loops can be closed. We plot this "cleaned" boundary with the induced orientation shown by the colour map changing from blue to pink.

When `show_tear_points=True`, we compute the "tear" points of our projection for the parameter `a`; we set `a=2.5` as default. For a point in our pointcloud, take its neighbours in the 1-skeleton and compute the mean length of these edges, then compute the mean Euclidean distance between our projected point and the projected neighbours. If the ratio of the mean projected length and the mean pointcloud length is greater than `a`, we classify this point as a "tear" point. We plot these as red crosses. These indicate the points which the projection "tears" the original geometry along, in order to give a planar representation.

When `show_connections=True`, we plot edges between ("clean") boundary points when they correspond to connected points in the 1-skeleton. However if these points are within `connection_tol` points away from each other, we ignore these connections. This option helps with visualising what quotient identifications should be make and with choosing parameters for `plot_quotient`. 

When `show_pointcloud=True`, we plot our pointcloud data with the corresponding orientated boundary and the "tear" point information (if True). We plot the location of the base point of the projection with a green triangle. We also plot the edge connections between boundary points in the pointcloud (`connection_tol` does not apply here). This option requires that the dimension of our pointcloud data is three.

### Quotient

The method `plot_quotient` computes and plots the quotient identifications of our projection - this requires that the dimension of our projection is two. The dotted lines represent non-glued edges and the solid lines represent glued edges. Glued edges with the same colour map are glued together in the orientation specified by the gradient of the colouring. The parameter `alpha0` controls the alpha values of the plotted edges.

In [5]:
quotient_info = S_rp2.plot_quotient(c=rp2[:, 0], alpha=1, tol=2, quotient_tol=15, tol1=5, connection_tol=5, alpha0=0.8, show_pointcloud=False, **params)

The parameters `alpha, tol, connection_tol` are used in the same way as above. They control how we extract an "clean" boundary from our projection and which connections between boundary points are to be included. We call boundary points with such allowed connections - "short-circuit points". The aim of our algorithm is to partition our boundary points into glued edges ("short-edges") and non-glued edges ("non-short-edges"), and to identify how glued edges are attached together.

We initially identify these edges by the rule that if two short-circuit points are more than `quotient_tol` points apart in the orientated boundary, then we define the set of points comprising this interval as a non-short-edge. This divides the set of remaining boundary points into separated intervals which are then each classified as a short-edge.

However, in the case of the above parameters for `S_rp2.plot_quotient`, if we look at the connections in `S_rp2.show_boundary`, we see that this algorithm would identify the entire boundary as a short-edge contradicting the construction of the real projective plane. This possibility necessitates the need to refine our short-edges to catch the seperate gluings within such short-edges 

To refine a short-edge, we look at the short-circuit points in the edge which have connections to other points in the same short-edge. We want to take the short-circuit point with the highest number of such connections and use these points to split apart the edge into several short-edges (think of the construction of the torus and the corner points). However looking at `S_rp2.show_boundary`, we see that due to the randomness in our data, we have a short-circuit point with two such connections whereas a single connection is necessary and sufficient. This results in the extra condition that such connections need to be spaced further apart than `tol1` points.

The details for the remainder of our quotient algorithm can be found in [[1]](#1)

When `show_pointcloud=True`, we also plot the original pointcloud with the corresponding glued and non-glued edges - this requires that the dimension of our pointcloud data is three.

### Other Examples

We demonstrate the other cases that `qrml` is built to handle. We use the examples of a sphere and truncated sphere.

##### Sphere

In [6]:
S_sphere = qrml.Simplex()
S_sphere.build_simplex(sphere, **params)
S_sphere.normal_coords(two_d=False, **params)

In [7]:
S_sphere.show_boundary(alpha=1, tol=1, c=sphere[:, 0], show_pointcloud=True, **params)  # uncleaned boundary

In [8]:
S_sphere.show_boundary(alpha=1, tol=10, c=sphere[:, 0], show_pointcloud=True, **params)  # cleaned boundary

The above cells shows the difference between the original boundary and "cleaned" boundary on the original pointcloud. We note that increasing `tol` past 10 would eventually result in this loop being closed leading to no boundary. This corresponds to the case where all points on the boundary should be identified together - e.g. the projection of the sphere rips points opposite the base point apart to display the pointcloud on the plane.

In [9]:
_ = S_sphere.plot_quotient(c=sphere[:, 0], alpha=1, tol=15, quotient_tol=15, tol1=5, alpha0=0.3, show_pointcloud=True)

We represent this case by plotting the boundary with a black line.

##### Truncated Sphere

In [10]:
trunc_sphere = sphere[sphere[:, 2]<=0.8]

S_trunc_sphere = qrml.Simplex()
S_trunc_sphere.build_simplex(trunc_sphere, **params)
S_trunc_sphere.normal_coords(two_d=False, **params)

In [11]:
_ = S_trunc_sphere.plot_quotient(c=trunc_sphere[:, 0], alpha=1, tol=15, quotient_tol=15, tol1=5, show_pointcloud=True)

In the case of a single non-short-edge (and no short-edges), we plot this single edge with a solid line colour, graded by the orientation we put on the loop.  

#### References

<a id="1">[1]</a> 
TODO

<a id="2">[2]</a> 
Tong, L., Zha, H. Riemannian manifold learning. *IEEE Transactions on Pattern Analysis
and Machine Intelligence* 30.5 (2008): 796-809.

<a id="3">[3]</a>
Alpha shape, *Wikipedia*, viewed 30 August 2022,
<https://en.wikipedia.org/wiki/Alpha_shape>

<a id="4">[4]</a> 
Hanniel, I. 2018, Calculate bounding polygon of alpha shape from the Delaunay triangulation, *Stack Overflow*, viewed 30 August 2022,  
<https://stackoverflow.com/questions/23073170/calculate-bounding-polygon-of-alpha-shape-from-the-delaunay-triangulation>